# Exercise 4: Linear MPC

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import solve_discrete_are
import cvxpy as cvx
from tqdm.auto import tqdm

# Constant problem variables
n, m = 2, 1
A = np.array([[1., 1.], [0., 1.]])
B = np.array([[0.], [1.]])
Q = np.eye(n)
R = 10.*np.eye(m)

In [ ]:
def solve_mpc(x0: np.ndarray, A: np.ndarray, B: np.ndarray,
           Qf: np.ndarray, Q: np.ndarray, R: np.ndarray,
           N: int, x_upper: np.ndarray, x_lower: np.ndarray, u_bound: float,
           rf: float) -> tuple[np.ndarray, np.ndarray, str]:
    """
    Solve the MPC problem starting at state `x0`.
    """
    n, m = Q.shape[0], R.shape[0]
    x_cvx = cvx.Variable((N + 1, n))
    u_cvx = cvx.Variable((N, m))

    ############################## Code starts here ##############################
    # INSTRUCTIONS: Construct and solve the MPC problem using CVXPY.

    ############################## Code ends here ##############################

    prob = cvx.Problem(cvx.Minimize(cost), constraints)
    prob.solve()
    x = x_cvx.value
    u = u_cvx.value
    status = prob.status

    return x, u, status

def simulate(x0, T, N, x_upper, x_lower, u_bound, Qf, rf, title=None):
    x = np.copy(x0)
    x_mpc = np.zeros((T, N + 1, n))
    u_mpc = np.zeros((T, N, m))
    cost = 0.
    success = True
    for t in range(T):
        x_mpc[t], u_mpc[t], status = solve_mpc(x, A, B, Qf, Q, R, N,
                                               x_upper, x_lower, u_bound, rf)
        u = u_mpc[t, 0, :]
        
        if status == 'infeasible':
            x_mpc = x_mpc[:t]
            u_mpc = u_mpc[:t]
            cost += np.inf
            if t == 0:
                success = False
                print("Failed to find initial feasible solution")
            break
        else:
            cost += x.T @ Q @ x + u.T @ R @ u
        x = A @ x + B @ u
    print(f"{title}: Cost = {cost:.2f}")
    if success:
        plot(title, x_mpc, u_mpc, x_upper, x_lower, u_bound)

def plot(title, x_mpc, u_mpc, x_upper, x_lower, u_bound):
    fig, ax = plt.subplots(1, 2, dpi=150, figsize=(10, 5))
    T = x_mpc.shape[0]
    ax[0].plot(x_mpc[:, 0, 0], x_mpc[:, 0, 1], '-o')
    for t in range(T):
        ax[0].plot(x_mpc[t, :, 0], x_mpc[t, :, 1], '--*', color='r', alpha=0.5, linewidth=0.5)
    ax[1].plot(u_mpc[:, 0], '-o')

    # Plot constraints
    ax[0].plot([x_upper[0], x_upper[0]], [x_lower[1], x_upper[1]], 'k--', linewidth=0.5)
    ax[1].plot([0, T], [u_bound, u_bound], 'k--', linewidth=0.5)
    ax[1].plot([0, T], [-u_bound, -u_bound], 'k--', linewidth=0.5)

    # Labels
    if title is not None:
        ax[0].set_title(title)
        ax[1].set_title(title)
    ax[0].set_xlabel(r'$x_{k,1}$')
    ax[1].set_xlabel(r'$k$')
    ax[0].set_ylabel(r'$x_{k,2}$')
    ax[1].set_ylabel(r'$u_k$')
    plt.show()

In [ ]:
N = 3 # mpc horizon
T = 20 # simulation time
x_upper = np.array([1., 5.])
x_lower = np.array([-6., -5.])
u_bound = 1.0 # control constraint
rf = np.inf # no terminal constraint
Qf = Q # terminal cost matrix
x0 = np.array([-5.5, 3.])

# Simulate the system
simulate(x0, T, N, x_upper, x_lower, u_bound, Qf, rf, title="Qf = Q")

In [ ]:
N = 3 # mpc horizon
T = 20 # simulation time
rf = np.inf # no terminal constraint
Qf = solve_discrete_are(A, B, Q, R)

# Simulate the system
simulate(x0, T, N, x_upper, x_lower, u_bound, Qf, rf, title="DARE")

In [ ]:
N = 3 # mpc horizon
T = 20 # simulation time
rf = 0.0 # add terminal constraint
Qf = solve_discrete_are(A, B, Q, R)

# Simulate the system
simulate(x0, T, N, x_upper, x_lower, u_bound, Qf, rf, title="Terminal constraint")